
Покращення MNIST за допомогою згорткових шарів

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# Завантажте дані
### START CODE HERE
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data()
### END CODE HERE

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# Обробка даних
# grader-required-cell
# GRADED FUNCTION: reshape_and_normalize
def reshape_and_normalize(images):
    ### START CODE HERE
    # Змініть форму зображень, щоб додати додатковий вимір
    images = images.reshape((images.shape[0], images.shape[1], images.shape[2], 1))
    # Нормалізуйте значення пікселів
    images = images / 255.0
    ### END CODE HERE
    return images

In [5]:
# Перевірте функцію
# Перезавантажте зображення у випадку багаторазового запуску цього рядка
(training_images, _), _ = tf.keras.datasets.mnist.load_data()
# Застосуйте вашу функцію
training_images = reshape_and_normalize(training_images)
print(f"Максимальне значення пікселя після нормалізації: {np.max(training_images)}\n")
print(f"Форма тренувального набору після зміни форми: {training_images.shape}\n")
print(f"Форма одного зображення після зміни форми: {training_images[0].shape}")

Максимальне значення пікселя після нормалізації: 1.0

Форма тренувального набору після зміни форми: (60000, 28, 28, 1)

Форма одного зображення після зміни форми: (28, 28, 1)


In [6]:
# Визначте зворотний виклик
# grader-required-cell
# GRADED CLASS: myCallback
### START CODE HERE
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.995:
            print("\nДосягнуто 99.5% точності, тому скасовуємо навчання!")
            self.model.stop_training = True
### END CODE HERE

In [7]:
# Визначте функцію моделі
# grader-required-cell
# GRADED FUNCTION: convolutional_model
def convolutional_model():
    ### START CODE HERE
    # Визначте модель
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    ### END CODE HERE
    # Скомпілюйте модель
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [8]:
# Збережіть ненавчену модель
model = convolutional_model()
# Отримайте кількість ваг
model_params = model.count_params()
# Тест на обмеження розміру моделі
assert model_params < 1000000, (
    f'Ваша модель має {model_params:,} параметрів. Для успішного оцінювання будь ласка, зберігайте її '
    f'менше 1 000 000, зменшуючи кількість одиниць у ваших шарах Conv2D та/або Dense.'
)
# Створити екземпляр класу зворотного виклику
callbacks = myCallback()
# Тренуйте модель (це може зайняти до 5 хвилин)
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])


Epoch 1/10
1875/1875 [==============================] - 52s 27ms/step - loss: 0.1477 - accuracy: 0.9563
Epoch 2/10
1875/1875 [==============================] - 46s 25ms/step - loss: 0.0512 - accuracy: 0.9845
Epoch 3/10
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0327 - accuracy: 0.9901
Epoch 4/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0217 - accuracy: 0.9930
Epoch 5/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0152 - accuracy: 0.9950
Epoch 6/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0104 - accuracy: 0.9967
Досягнуто 99.5% точності, тому скасовуємо навчання!
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0104 - accuracy: 0.9966


In [10]:


# grader-required-cell
print(f"Вашу модель навчено протягом {len(history.epoch)} епох")

Вашу модель навчено протягом 6 епох


In [11]:

if not "accuracy" in history.model.metrics_names:
    print("Використовуйте 'accuracy' як метрику при компіляції вашої моделі.")
else:
    print("Метрика була визначена правильно.")

Метрика була визначена правильно.
